# 神经网络框架的基本原理

## 实现原理

一个普通的神经网络运算流程，主要包括：

- 输入数据
- 网络层前向传播
- 计算损失
- 网络层反向传播梯度
- 更新参数

这里将神经网络抽象为4个组件：
- 数据输入：一般构建为tensor张量
- 计算层（包括激活函数）：负责接收上一层输入，将结果输出给下一层，实现向前和向后两个方向的计算
- 损失计算：给定预测值和真实值后，使用该组件计算损失值和最后一层梯度
- 优化器：使用求解后的梯度更新模型的参数矩阵

## 另一个模型定义

In [11]:
import os
import torch
import numpy as np
from tqdm import tqdm
from torchvision import datasets, transforms

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

batch_size = 320
epochs = 1024

#device = "cuda"
device = "cpu"

# 创建一个三层全连接网络
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(28*28, 312),
            torch.nn.ReLU(),
            torch.nn.Linear(312, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 10),
        )
    def forward(self, input):
        x = self.faltten(input)
        logits = self.linear_relu_stack(x)
        return logits

# 加载模型
model = NeuralNetwork()
model = model.to(device)
model = torch.compile(model)

# 损失函数
loss_fu = torch.nn.CrossEntropyLoss()

# 求解器
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 数据预处理（转tensor、数值归一化）
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 自动下载MNIST数据集
dataset_train = datasets.MNIST('data', train=True, download=True, transform=transform)
dataset_test = datasets.MNIST('data', train=False, transform=transform)

# 定义数据加载器（自动对数据加载、多线程、随机化、划分batch、等等）
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size)

In [13]:
print(dataset_train)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )